# Hand-Crafted Prompts Testing Playground
This notebook is made to offer means for testing the hand-crafted prompting method on a selection of LLMs. The notebook is meant to be a plug-and-play manner, where you could set the model id and run the cells to generate results directly.

### Load Model and Tokenizer

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

### Setting the Prompt Template
Based on Plaza-Del-Arco, F., Nozza, D., & Hovy, D. (2023). Respectful or Toxic? Using Zero-Shot Learning with Language Models to Detect Hate Speech. Retrieved from https://aclanthology.org/2023.woah-1.6.pdf

In [43]:
prompt_template = "Classify this text as hate or non-hate. Text:"
output_indicator = "Answer:"

### Loading Test Data

In [11]:
import pandas as pd

In [44]:
df = pd.read_csv('./data/Dynabench/preprocessed_data.csv')
df.head()

,class,tweet
0,2,as a woman you shouldnt complain about cleanin...
1,1,boy dats coldtyga dwn bad for cuffin dat hoe i...
2,1,dawg you ever fuck a bitch and she start to cr...
3,1,she look like a tranny
4,1,the shit you hear about me might be true or it...


In [45]:
labels = {
    "0": "Hate Speech",
    "1": "Offensive",
    "2": "Non-Hate"
}

### Concatenate Prompt Template to Input Samples

In [46]:
df['input'] = df['tweet'].apply(lambda x: f"{prompt_template} {x}. {output_indicator}")

In [47]:
text_data = df['input'].astype("str").tolist()
text_data

['Classify this text as hate or non-hate. Text: as a woman you shouldnt complain about cleaning up your house as a man you should always take the trash out. Answer:',
 'Classify this text as hate or non-hate. Text: boy dats coldtyga dwn bad for cuffin dat hoe in the st place. Answer:',
 'Classify this text as hate or non-hate. Text: dawg you ever fuck a bitch and she start to cry you be confused as shit. Answer:',
 'Classify this text as hate or non-hate. Text: she look like a tranny. Answer:',
 'Classify this text as hate or non-hate. Text: the shit you hear about me might be true or it might be faker than the bitch who told it to ya. Answer:',
 'Classify this text as hate or non-hate. Text: the shit just blows meclaim you so faithful and down for somebody but still fucking with hoes. Answer:',
 'Classify this text as hate or non-hate. Text: i can not just sit up and hate on another bitch i got too much shit going on. Answer:',
 'Classify this text as hate or non-hate. Text: cause im 

In [48]:
text_data_batch = text_data[0:10]

In [49]:
inputs = tokenizer(text_data_batch, return_tensors="pt", padding=True, truncation=True)

In [50]:
sequences = model.generate(**inputs, do_sample=True, min_length=5, max_length=10, temperature=1)

In [51]:
decoded = tokenizer.batch_decode(sequences, skip_special_tokens=True)
for i, out in enumerate(decoded):
    print(f"Sentence: {text_data_batch[i]}")
    print(out)
    print(f"Truth: ")
    print(labels[str(df["class"][i])])
    print("-" * 80)

Sentence: Classify this text as hate or non-hate. Text: as a woman you shouldnt complain about cleaning up your house as a man you should always take the trash out. Answer:
Non-hate
Truth: 
Non-Hate
--------------------------------------------------------------------------------
Sentence: Classify this text as hate or non-hate. Text: boy dats coldtyga dwn bad for cuffin dat hoe in the st place. Answer:
Hateumber
Truth: 
Offensive
--------------------------------------------------------------------------------
Sentence: Classify this text as hate or non-hate. Text: dawg you ever fuck a bitch and she start to cry you be confused as shit. Answer:
Hate or hate
Truth: 
Offensive
--------------------------------------------------------------------------------
Sentence: Classify this text as hate or non-hate. Text: she look like a tranny. Answer:
hate  hate
Truth: 
Offensive
--------------------------------------------------------------------------------
Sentence: Classify this text as hate o